# Automatic Model Selection

Test notebook for automatic model selection

In [1]:
import datetime
import logging
import sys
import numpy as np
import time
import math

from sklearn.preprocessing import StandardScaler, MinMaxScaler

from keras.callbacks import LearningRateScheduler
from keras.optimizers import Adam
import keras.backend as K

#sys.path.append('/Users/davidlaredorazo/Documents/University_of_California/Research/Projects')
sys.path.append('/media/controlslab/DATA/Projects')

import ann_framework.aux_functions as aux_functions

import automatic_model_selection
from automatic_model_selection import Configuration
from ann_encoding_rules import Layers
import fetch_to_keras
#from CMAPSAuxFunctions import TrainValTensorBoard

#Tunable model
from ann_framework.tunable_model.tunable_model import SequenceTunableModelRegression, SequenceTunableModelClassification

#Data handlers
from ann_framework.data_handlers.data_handler_CMAPSS import CMAPSSDataHandler
from ann_framework.data_handlers.data_handler_MNIST import MNISTDataHandler
from ann_framework.data_handlers.data_handler_CIFAR10 import CIFAR10DataHandler

learningRate_scheduler = LearningRateScheduler(aux_functions.step_decay)

size_scaler = 0.5

#Use same configuration for all experiments, just change some of the parameters

#Define some random paramaters for the creation of the configuration, this will change for each test model
architecture_type = Layers.FullyConnected
#architecture_type = Layers.Convolutional
problem_type = 2  #1 for regression, 2 for classification
output_shape = 10 #If regression applies, number of classes
input_shape = (784,)

config = Configuration(architecture_type, problem_type, input_shape, output_shape, pop_size=5, 
                       tournament_size=3, max_similar=3, epochs=5, cross_val=0.2, size_scaler=size_scaler,
                       max_generations=10, binary_selection=True, mutation_ratio=0.8, 
                       similarity_threshold=0.2, more_layers_prob=0.4, verbose_individuals=True, 
                       show_model=True, verbose_training=1)

Using TensorFlow backend.


## Given a model get the compiled model

In [2]:
def get_compiled_model(model, problem_type, optimizer_params=[]):
    """Obtain a keras compiled model"""
    
    #Shared parameters for the models
    optimizer = Adam(lr=0.01, beta_1=0.5)
    
    if problem_type == 1:
        lossFunction = "mean_squared_error"
        metrics = ["mse"]
    elif problem_type == 2:
        lossFunction = "categorical_crossentropy"
        metrics = ["accuracy"]
    else:
        print("Problem type not defined")
        model = None
        return
    
    #Create and compile the models
    model.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)
    
    return model


def create_tunable_model(model_genotype, problem_type, input_shape, data_handler, model_number):
    
    K.clear_session()
    
    model = fetch_to_keras.decode_genotype(model_genotype, problem_type, input_shape, 1)
    
    model = get_compiled_model(model, problem_type, optimizer_params=[])
    
    if problem_type == 1:
        tModel = SequenceTunableModelRegression('ModelReg_SN_'+str(model_number), model, lib_type='keras', data_handler=data_handler)
    else:
        tModel = SequenceTunableModelClassification('ModelClass_SN_'+str(model_number), model, lib_type='keras', data_handler=data_handler)
        
    return tModel

## Load the CMAPSS datahandler

In [3]:
def cmapss_dhandler():

    #Selected as per CNN paper
    features = ['T2', 'T24', 'T30', 'T50', 'P2', 'P15', 'P30', 'Nf', 'Nc', 'epr', 'Ps30', 'phi', 'NRf', 'NRc', 'BPR', 
    'farB', 'htBleed', 'Nf_dmd', 'PCNfR_dmd', 'W31', 'W32']
    selected_indices = np.array([2, 3, 4, 7, 8, 9, 11, 12, 13, 14, 15, 17, 20, 21])
    selected_features = list(features[i] for i in selected_indices-1)
    data_folder = '../CMAPSSData'

    window_size = 24
    window_stride = 1
    max_rul = 129

    dhandler_cmapss = CMAPSSDataHandler(data_folder, 1, selected_features, max_rul, window_size, window_stride)

    input_shape = (len(selected_features)*window_size, )

    return dhandler_cmapss, input_shape

## Function to save top models

In [4]:
def save_best_models(best_models_list, global_best_model_index, saveto, input_shape, data_handler, 
                     problem_type=1, data_scaler=None, train_epochs=100, metrics=[], round=0):
    
    n_models = len(best_models_list)
    
    for ind_model, i in zip(best_models_list, range(n_models)):
        
        tModel = create_tunable_model(ind_model.stringModel, problem_type, input_shape, data_handler, i)
        kmodel = tModel.model
        model_json = kmodel.to_json()
        
        #Save model's architecture
        string_append = str(i) if i != global_best_model_index else 'global'
        with open(saveto+"bestModel_"+string_append+".json", "w") as json_file:
            json_file.write(model_json)
            
    #Train the global best, model has to be recompiled
    ind_model = best_models_list[global_best_model_index]
    tModel = create_tunable_model(ind_model.stringModel, problem_type, input_shape, data_handler, n_models)
    
    print(tModel.model.summary())
    print(tModel.data_handler)
    
    if tModel.data_handler.data_scaler != None:
        print("Using data handler scaler")
    elif tModel.data_scaler != None:
        print("Using tModel scaler (Overriding data handler scaler)")
    else:
        print("No data scaling used")
    
    if data_scaler != None:
        tModel.data_handler.data_scaler = None
        tModel.data_scaler = data_scaler
        
    tModel.load_data(unroll=True, verbose=1, cross_validation_ratio=0)
    tModel.print_data()
    tModel.epochs = train_epochs

    tModel.train_model(verbose=1)
    
    tModel.evaluate_model(metrics, round=round)
    
    kmodel = tModel.model
            
    # serialize weights to HDF5
    kmodel.save_weights(saveto+"bestModel_global.h5")
    
    print("Saved models for dataset 1 to disk")

## Get global best model

In [5]:
def recompute_globals_fitness(best_models, size_scaler, problem_type):
    """It is necessary to recompute the fiteness of global models since they have differnt normalization factors"""

    #print("Before normalization")
    #automatic_model_selection.print_best(best_models)
    
    normalize_scores(best_models)
    
    #print("After normalization")
    #automatic_model_selection.print_best(best_models)
    
    global_best_index = compute_fitness(best_models, size_scaler, problem_type)
    
    print("Recomputed fitness")
    automatic_model_selection.print_best(best_models)
    print("Global best index")
    print(global_best_index)
    
    return global_best_index


def normalize_scores(best_models):
    
    pop_size = len(best_models)
    raw_scores = np.zeros((pop_size,))
    
    for i in range(pop_size):
        model = best_models[i]
        raw_scores[i] = model.raw_score
        
    normalization_factor = np.linalg.norm(raw_scores)
    normalized_scores = raw_scores/normalization_factor
    
    for i in range(pop_size):
        model = best_models[i]
        model.normalized_score = raw_scores[i]
    
    
def compute_fitness(best_models, size_scaler, problem_type):
    
    pop_size = len(best_models)
    
    global_best_index = 0
    
    for i in range(pop_size):
        
        round_up_to = 3

        #Round up to the first 3 digits before computing log                                                                                                                                                          
        rounding_scaler = 10**round_up_to
        trainable_count = round(best_models[i].raw_size/rounding_scaler)*rounding_scaler
        size_score = math.log10(trainable_count)

        scaled_score = best_models[i].normalized_score

        #For classification estimate the error which is between 0 and 1                                                                                                                   
        if problem_type == 2:
            metric_score = (1 - scaled_score)*10 #Multiply by 10 to have a better scaling. I still need to find an appropriate scaling                                                
        else:
            metric_score = scaled_score*10 #Multiply by 10 to have a better scaling. I still need to find an appropiate scaling                                                       
    
        metric_scaler = 1-size_scaler
        print("metric_scaler %f"%metric_scaler)
        print("size scaler %f"%size_scaler)
    
        #Scalarization of multiobjective version of the fitness function                                                                                                                  
        best_models[i].fitness = metric_scaler*metric_score + size_scaler*size_score
        
        if best_models[i].fitness < best_models[global_best_index].fitness:
            global_best_index = i
            
    return global_best_index

## Test on MNIST

In [6]:
def mnist_test(dhandler_mnist, size_scaler=0.5, total_experiments=3):

    """Input can be of 3 types, ANN (1), CNN (2) or RNN (3)"""
    architecture_type = Layers.FullyConnected
    #architecture_type = Layers.Convolutional
    problem_type = 2  #1 for regression, 2 for classification
    output_shape = 10 #If regression applies, number of classes
    input_shape = (784,)
    #input_shape = (28,28,1)

    """
    pop_size = 5
    tournament_size = 3
    max_similar = 3
    total_experiments = 5
    count_experiments = 0
    unroll = True
    """
    #total_experiments = 1
    count_experiments = 0
    unroll = True

    global_best_list = []
    global_best = None
    global_best_index = 0
    experiment_times = np.zeros((total_experiments,1))

    scaler = None

    t = datetime.datetime.now()
    
    #Clear logging facility before attempting to create log
    for handler in logging.root.handlers[:]:
        logging.root.removeHandler(handler)
    
    logging.basicConfig(filename='logs/nn_evolution_mnist_' + t.strftime('%m%d%Y%H%M%S') + '.log', level=logging.INFO, 
                            format='%(levelname)s:%(threadName)s:%(message)s', datefmt='%m/%d/%Y %H:%M:%S')

    #mnist datahandler
    #dHandler_mnist = MNISTDataHandler()

    config.architecture_type = architecture_type
    config.problem_type = problem_type
    config.input_shape = input_shape
    config.output_shape = output_shape
    config.size_scaler = size_scaler
    config.epochs=5

    """
    config = Configuration(architecture_type, problem_type, input_shape, output_shape, pop_size, 
                           tournament_size, max_similar, epochs=20, cross_val=0.2, size_scaler=size_scaler,
                           max_generations=10, binary_selection=True, mutation_ratio=0.4, 
                           similarity_threshold=0.2, more_layers_prob=0.8)
    """

    while count_experiments < total_experiments:
        print("Launching experiment {}".format(count_experiments+1))
        logging.info("Launching experiment {}".format(count_experiments+1))


        start = time.time()
        best = automatic_model_selection.run_experiment(config, dHandler_mnist, count_experiments + 1, unroll=unroll,
                                                        learningRate_scheduler=learningRate_scheduler, 
                                                        tModel_scaler=scaler)
        end = time.time()
        elapsed_time = (end-start)/60
        experiment_times[count_experiments] = elapsed_time
        print("Experiment time: {} minutes".format(elapsed_time))
        logging.info("Experiment time: {} minutes".format(elapsed_time))


        best.individual_label = count_experiments

        global_best_list.append(best)
        
        """
        if global_best == None:
            global_best = best
        else:
            if best.fitness < global_best.fitness:
                global_best = best
                global_best_index = count_experiments
        """

        count_experiments =  count_experiments + 1
    
    print("Recompute globals fitness")
    global_best_index = recompute_globals_fitness(global_best_list, config.size_scaler, config.problem_type)
    global_best = global_best_list[global_best_index]
    
    total_experiment_time = experiment_times.sum()

    print("Global best list\n")
    logging.info("Global best list\n")
    automatic_model_selection.print_best(global_best_list)

    print("Global best is\n")
    print(global_best)
    logging.info("Global best is\n")
    logging.info(global_best)

    print("Global time {}".format(experiment_times.sum()))
    logging.info("Global time {}".format(experiment_times.sum()))
    
    logging.shutdown()
    
    return global_best_list, global_best_index, total_experiment_time

## Test on CIFAR10

In [7]:
def cifar_test(dHandler_cifar, size_scaler=0.5, total_experiments=3):

    """Input can be of 3 types, ANN (1), CNN (2) or RNN (3)"""
    architecture_type = Layers.FullyConnected
    problem_type = 2  #1 for regression, 2 for classification
    output_shape = 10 #If regression applies, number of classes
    input_shape = (3072,)
    """
    pop_size = 5
    tournament_size = 3
    max_similar = 3
    """
    total_experiments = 5
    count_experiments = 0
    unroll = True

    global_best_list = []
    global_best = None
    global_best_index = 0

    scaler = None

    t = datetime.datetime.now()

    logging.basicConfig(filename='logs/nn_evolution_cifar10_' + t.strftime('%m%d%Y%H%M%S') + '.log', level=logging.INFO, 
                            format='%(levelname)s:%(threadName)s:%(message)s', datefmt='%m/%d/%Y %H:%M:%S')

    #mnist datahandler
    #dHandler_cifar = CIFAR10DataHandler()

    """
    config = Configuration(architecture_type, problem_type, input_shape, output_shape, pop_size, tournament_size, max_similar, 
                           epochs=5, cross_val=0.2, size_scaler=size_scaler, max_generations=10, binary_selection=True, 
                           mutation_ratio=0.4, similarity_threshold=0.2, more_layers_prob=0.8)
    """

    config.architecture_type = architecture_type
    config.problem_type = problem_type
    config.input_shape = input_shape
    config.output_shape = output_shape

    while count_experiments < total_experiments:
        print("Launching experiment {}".format(count_experiments+1))
        logging.info("Launching experiment {}".format(count_experiments+1))

        best = automatic_model_selection.run_experiment(config, dHandler_cifar, count_experiments + 1, unroll=unroll,
                                                        learningRate_scheduler=learningRate_scheduler, 
                                                        tModel_scaler=scaler, verbose_data=0)

        best.individual_label = count_experiments

        global_best_list.append(best)

        if global_best == None:
            global_best = best
        else:
            if best.fitness < global_best.fitness:
                global_best = best
                global_best_index = count_experiments

        count_experiments =  count_experiments + 1
        
    total_experiment_time = experiment_times.sum()

    print("Global best list\n")
    logging.info("Global best list\n")
    automatic_model_selection.print_best(global_best_list)

    print("Global best is\n")
    print(global_best)
    logging.info("Global best is\n")
    logging.info(global_best)
    
    return global_best_list, global_best_index, total_experiment_time

## Test on CMAPSS

In [8]:
def cmapss_test(dhandler_cmapss, input_shape, size_scaler=0.5, total_experiments=3):

    """Input can be of 3 types, ANN (1), CNN (2) or RNN (3)"""
    architecture_type = Layers.FullyConnected
    problem_type = 1  #1 for regression, 2 for classification
    output_shape = 1 #If regression applies, number of classes

    """
    pop_size = 5
    tournament_size = 3
    max_similar = 3
    """
    #total_experiments = 5
    count_experiments = 0
    unroll = True

    global_best_list = []
    global_best = None
    global_best_index = 0
    experiment_times = np.zeros((total_experiments,1))

    scaler = MinMaxScaler(feature_range=(-1, 1))

    t = datetime.datetime.now()

    logging.basicConfig(filename='logs/nn_evolution_cmapss_' + t.strftime('%m%d%Y%H%M%S') + '.log', level=logging.INFO, 
                            format='%(levelname)s:%(threadName)s:%(message)s', datefmt='%m/%d/%Y %H:%M:%S')

    #cmaps datahandler
    #dhandler_cmaps, input_shape = cmaps_dhandler()

    """
    config = Configuration(architecture_type, problem_type, input_shape, output_shape, pop_size, tournament_size, 
                           max_similar, epochs=5, cross_val=0.2, size_scaler=size_scaler, max_generations=10, 
                           binary_selection=True, mutation_ratio=0.4, similarity_threshold=0.2, more_layers_prob=0.8)
    """

    config.architecture_type = architecture_type
    config.problem_type = problem_type
    config.input_shape = input_shape
    config.output_shape = output_shape
    config.epochs=10

    while count_experiments < total_experiments:
        print("Launching experiment {}".format(count_experiments+1))
        logging.info("Launching experiment {}".format(count_experiments+1))

        start = time.time()
        best = automatic_model_selection.run_experiment(config, dhandler_cmapss, count_experiments + 1, unroll=unroll,
                                                        learningRate_scheduler=learningRate_scheduler, 
                                                        tModel_scaler=scaler)
        end = time.time()
        elapsed_time = (end-start)/60
        experiment_times[count_experiments] = elapsed_time
        print("Experiment time: {} minutes".format(elapsed_time))
        logging.info("Experiment time: {} minutes".format(elapsed_time))

        best.individual_label = count_experiments

        global_best_list.append(best)

        """
        if global_best == None:
            global_best = best
        else:
            if best.fitness < global_best.fitness:
                global_best = best
                global_best_index = count_experiments
        """

        count_experiments =  count_experiments + 1
        
    print("Recompute globals fitness")
    global_best_index = recompute_globals_fitness(global_best_list, config.size_scaler, config.problem_type)
    global_best = global_best_list[global_best_index]
        
    total_experiment_time = experiment_times.sum()

    print("Global best list\n")
    logging.info("Global best list\n")
    automatic_model_selection.print_best(global_best_list)

    print("Global best is\n")
    print(global_best)
    logging.info("Global best is\n")
    logging.info(global_best)

    print("Global time {}".format(total_experiment_time))
    logging.info("Global time {}".format(total_experiment_time))
    
    return global_best_list, global_best_index, total_experiment_time

## Perform tests

In [9]:
"""

#alphas = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]
alphas = [0.5]
experiments = 5

global_best_list = []
global_best_index = 0
total_experiment_time = []
total_experiment_time = 0
avg_experiment_time = 0

dHandler_mnist = MNISTDataHandler()

for size_scaler in alphas:

    print("Running for alpha={}".format(size_scaler))
    
    global_best_list, global_best_index, total_experiment_time = mnist_test(dHandler_mnist, 
                                                                            size_scaler=size_scaler, 
                                                                            total_experiments=experiments)
    
    avg_experiment_time = total_experiment_time/experiments
    
    print("Total experiment time {}".format(total_experiment_time))
    print("Avg experiment time {}".format(avg_experiment_time))
    
    save_best_models(global_best_list, global_best_index, 
                     'best_models/mnist/yulin/alpha{}/'.format(size_scaler), input_shape=input_shape, 
                     data_handler=dHandler_mnist, problem_type=problem_type, train_epochs=100)

"""

'\n\n#alphas = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]\nalphas = [0.5]\nexperiments = 5\n\nglobal_best_list = []\nglobal_best_index = 0\ntotal_experiment_time = []\ntotal_experiment_time = 0\navg_experiment_time = 0\n\ndHandler_mnist = MNISTDataHandler()\n\nfor size_scaler in alphas:\n\n    print("Running for alpha={}".format(size_scaler))\n    \n    global_best_list, global_best_index, total_experiment_time = mnist_test(dHandler_mnist, \n                                                                            size_scaler=size_scaler, \n                                                                            total_experiments=experiments)\n    \n    avg_experiment_time = total_experiment_time/experiments\n    \n    print("Total experiment time {}".format(total_experiment_time))\n    print("Avg experiment time {}".format(avg_experiment_time))\n    \n    save_best_models(global_best_list, global_best_index, \n                     \'best_models/mnist/yulin/alpha{}/\'.format(size_scaler)

In [10]:
""""
dHandler_cifar = CIFAR10DataHandler()

global_best_list, global_best_index = cifar_test(dHandler_cifar, size_scaler=size_scaler, total_experiments=1)
"""

'"\ndHandler_cifar = CIFAR10DataHandler()\n\nglobal_best_list, global_best_index = cifar_test(dHandler_cifar, size_scaler=size_scaler, total_experiments=1)\n'

In [11]:
#alphas = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]
alphas = [0.6]
experiments = 5
problem_type = 1

global_best_list = []
global_best_index = 0
total_experiment_time = []
total_experiment_time = 0
avg_experiment_time = 0

scaler = MinMaxScaler(feature_range=(-1, 1))

dhandler_cmapss, input_shape = cmapss_dhandler()
dhandler_cmapss_for_best, input_shape = cmapss_dhandler()

for size_scaler in alphas:

    print("Running for alpha={}".format(size_scaler))
    
    global_best_list, global_best_index, total_experiment_time = cmapss_test(dhandler_cmapss=dhandler_cmapss, 
                                                                            input_shape=input_shape,
                                                                            size_scaler=size_scaler, 
                                                                            total_experiments=experiments)
    
    print(global_best_list)
    print(global_best_index)
    
    avg_experiment_time = total_experiment_time/experiments
    
    print("Total experiment time {}".format(total_experiment_time))
    print("Avg experiment time {}".format(avg_experiment_time))
    
    save_best_models(global_best_list, global_best_index, 
                     'best_models/cmapss/yulin/alpha{}/'.format(size_scaler), input_shape=input_shape, 
                     data_handler=dhandler_cmapss_for_best, problem_type=problem_type, train_epochs=100, 
                     data_scaler=scaler)
    
    

Running for alpha=0.6
Launching experiment 1

Generation 1
launch new
True
gen similar
False
Fetching model 0 to keras
Evaluating model 0
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
in (Dense)                   (None, 760)               256120    
_________________________________________________________________
dropout_1 (Dropout)          (None, 760)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 761       
Total params: 256,881
Trainable params: 256,881
Non-trainable params: 0
_________________________________________________________________
None
Loading data for the first time
Reloading data due to parameter change
Loading data for dataset 1 with window_size of 24, stride of 1 and maxRUL of 129. Cros-Validation ratio 0.2
Loading data from file and computing dataframes
training with cv
Train on 1461

14613/14613 [==============================] - 0s 3us/step - loss: 763.0258 - mean_squared_error: 763.0258 - val_loss: 1344.9788 - val_mean_squared_error: 1344.9788
Epoch 10/10
20/20 [==============================] - 0s 19us/step
Fetching model 3 to keras
Evaluating model 3
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
in (Dense)                   (None, 656)               221072    
_________________________________________________________________
dropout_1 (Dropout)          (None, 656)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                21024     
_________________________________________________________________
dense_2 (Dense)              (None, 864)               28512     
_________________________________________________________________
dropout_2 (Dropout)          (None, 864)               0        

Train on 14613 samples, validate on 20 samples
Epoch 1/10
14613/14613 [==============================] - 0s 10us/step - loss: 6794.5818 - mean_squared_error: 6794.5818 - val_loss: 5066.1416 - val_mean_squared_error: 5066.1416
Epoch 2/10
14613/14613 [==============================] - 0s 3us/step - loss: 4162.9396 - mean_squared_error: 4162.9396 - val_loss: 3355.0085 - val_mean_squared_error: 3355.0085
Epoch 3/10
14613/14613 [==============================] - 0s 3us/step - loss: 2550.3792 - mean_squared_error: 2550.3792 - val_loss: 2278.9583 - val_mean_squared_error: 2278.9583
Epoch 4/10
14613/14613 [==============================] - 0s 3us/step - loss: 1586.2899 - mean_squared_error: 1586.2899 - val_loss: 1681.2920 - val_mean_squared_error: 1681.2920
Epoch 5/10
14613/14613 [==============================] - 0s 3us/step - loss: 1072.7742 - mean_squared_error: 1072.7742 - val_loss: 1375.6278 - val_mean_squared_error: 1375.6278
Epoch 6/10
14613/14613 [==============================] - 0s 3

14613/14613 [==============================] - 0s 12us/step - loss: 7356.9635 - mean_squared_error: 7356.9635 - val_loss: 5990.3521 - val_mean_squared_error: 5990.3521
Epoch 2/10
14613/14613 [==============================] - 0s 3us/step - loss: 5506.9356 - mean_squared_error: 5506.9356 - val_loss: 4703.2749 - val_mean_squared_error: 4703.2749
Epoch 3/10
14613/14613 [==============================] - 0s 3us/step - loss: 4182.9446 - mean_squared_error: 4182.9446 - val_loss: 3693.3938 - val_mean_squared_error: 3693.3938
Epoch 4/10
14613/14613 [==============================] - 0s 3us/step - loss: 3145.8894 - mean_squared_error: 3145.8894 - val_loss: 2917.3872 - val_mean_squared_error: 2917.3872
Epoch 5/10
14613/14613 [==============================] - 0s 3us/step - loss: 2353.9695 - mean_squared_error: 2353.9695 - val_loss: 2333.9551 - val_mean_squared_error: 2333.9551
Epoch 6/10
14613/14613 [==============================] - 0s 3us/step - loss: 1776.3473 - mean_squared_error: 1776.3473 

training with cv
Train on 14613 samples, validate on 20 samples
Epoch 1/10
14613/14613 [==============================] - 0s 12us/step - loss: 4825.5034 - mean_squared_error: 4825.5034 - val_loss: 1501.2100 - val_mean_squared_error: 1501.2100
Epoch 2/10
14613/14613 [==============================] - 0s 3us/step - loss: 953.8172 - mean_squared_error: 953.8172 - val_loss: 918.2794 - val_mean_squared_error: 918.2794
Epoch 3/10
14613/14613 [==============================] - 0s 3us/step - loss: 499.3601 - mean_squared_error: 499.3601 - val_loss: 838.8182 - val_mean_squared_error: 838.8182
Epoch 4/10
14613/14613 [==============================] - 0s 3us/step - loss: 422.7506 - mean_squared_error: 422.7506 - val_loss: 799.1083 - val_mean_squared_error: 799.1083
Epoch 5/10
14613/14613 [==============================] - 0s 3us/step - loss: 387.0418 - mean_squared_error: 387.0418 - val_loss: 760.4734 - val_mean_squared_error: 760.4734
Epoch 6/10
14613/14613 [==============================] - 0s 

...

training with cv
Train on 14613 samples, validate on 20 samples
Epoch 1/10
14613/14613 [==============================] - 0s 12us/step - loss: 5417.2625 - mean_squared_error: 5417.2625 - val_loss: 1070.1282 - val_mean_squared_error: 1070.1282
Epoch 2/10
14613/14613 [==============================] - 0s 3us/step - loss: 599.6964 - mean_squared_error: 599.6964 - val_loss: 869.3070 - val_mean_squared_error: 869.3070
Epoch 3/10
14613/14613 [==============================] - 0s 3us/step - loss: 417.8834 - mean_squared_error: 417.8834 - val_loss: 765.5420 - val_mean_squared_error: 765.5420
Epoch 4/10
14613/14613 [==============================] - 0s 3us/step - loss: 332.4763 - mean_squared_error: 332.4763 - val_loss: 704.4553 - val_mean_squared_error: 704.4553
Epoch 5/10
14613/14613 [==============================] - 0s 3us/step - loss: 296.1604 - mean_squared_error: 296.1604 - val_loss: 691.1417 - val_mean_squared_error: 691.1417
Epoch 6/10
14613/14613 [==============================] - 0s 

training with cv
Train on 14613 samples, validate on 20 samples
Epoch 1/10
14613/14613 [==============================] - 0s 11us/step - loss: 7836.5386 - mean_squared_error: 7836.5386 - val_loss: 6346.2061 - val_mean_squared_error: 6346.2061
Epoch 2/10
14613/14613 [==============================] - 0s 3us/step - loss: 5558.4408 - mean_squared_error: 5558.4408 - val_loss: 4233.0195 - val_mean_squared_error: 4233.0195
Epoch 3/10
14613/14613 [==============================] - 0s 3us/step - loss: 3452.3630 - mean_squared_error: 3452.3630 - val_loss: 2601.0991 - val_mean_squared_error: 2601.0991
Epoch 4/10
14613/14613 [==============================] - 0s 3us/step - loss: 2177.1340 - mean_squared_error: 2177.1340 - val_loss: 1733.1282 - val_mean_squared_error: 1733.1282
Epoch 5/10
14613/14613 [==============================] - 0s 3us/step - loss: 1588.7011 - mean_squared_error: 1588.7011 - val_loss: 1370.8528 - val_mean_squared_error: 1370.8528
Epoch 6/10
14613/14613 [=====================

14613/14613 [==============================] - 0s 11us/step - loss: 7670.2645 - mean_squared_error: 7670.2645 - val_loss: 5983.9326 - val_mean_squared_error: 5983.9326
Epoch 2/10
14613/14613 [==============================] - 0s 3us/step - loss: 4854.7474 - mean_squared_error: 4854.7474 - val_loss: 3414.1602 - val_mean_squared_error: 3414.1602
Epoch 3/10
14613/14613 [==============================] - 0s 3us/step - loss: 2590.1348 - mean_squared_error: 2590.1348 - val_loss: 1947.8398 - val_mean_squared_error: 1947.8398
Epoch 4/10
14613/14613 [==============================] - 0s 3us/step - loss: 1663.4508 - mean_squared_error: 1663.4508 - val_loss: 1429.0084 - val_mean_squared_error: 1429.0084
Epoch 5/10
14613/14613 [==============================] - 0s 3us/step - loss: 1298.4178 - mean_squared_error: 1298.4178 - val_loss: 1212.3381 - val_mean_squared_error: 1212.3381
Epoch 6/10
14613/14613 [==============================] - 0s 3us/step - loss: 1016.0399 - mean_squared_error: 1016.0399 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
in (Dense)                   (None, 184)               62008     
_________________________________________________________________
dense_1 (Dense)              (None, 184)               34040     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 185       
Total params: 96,233
Trainable params: 96,233
Non-trainable params: 0
_________________________________________________________________
None
No data scaling used
Loading data for the first time
Reloading data due to parameter change
Loading data for dataset 1 with window_size of 24, stride of 1 and maxRUL of 129. Cros-Validation ratio 0
Loading data from file and computing dataframes
Printing shapes

Training data (X, y)
(18331, 336)
(18331, 1)
Testing data (X, y)
(100, 336)
(100, 1)
Printing first 5 elements

Training data (X, y)
[

18331/18331 [==============================] - 0s 3us/step - loss: 130.8497 - mean_squared_error: 130.8497
Epoch 52/100
18331/18331 [==============================] - 0s 3us/step - loss: 130.0345 - mean_squared_error: 130.0345
Epoch 53/100
18331/18331 [==============================] - 0s 3us/step - loss: 129.3206 - mean_squared_error: 129.3206
Epoch 54/100
18331/18331 [==============================] - 0s 3us/step - loss: 133.6997 - mean_squared_error: 133.6997
Epoch 55/100
18331/18331 [==============================] - 0s 3us/step - loss: 125.3525 - mean_squared_error: 125.3525
Epoch 56/100
18331/18331 [==============================] - 0s 3us/step - loss: 124.2852 - mean_squared_error: 124.2852
Epoch 57/100
18331/18331 [==============================] - 0s 3us/step - loss: 120.2450 - mean_squared_error: 120.2450
Epoch 58/100
18331/18331 [==============================] - 0s 3us/step - loss: 109.1363 - mean_squared_error: 109.1363
Epoch 59/100
18331/18331 [===========================

In [12]:
"""
save_best_models(global_best_list, global_best_index, 'best_models/mnist/alpha{}/scalarized_version/'.format(size_scaler), 
                 input_shape=input_shape, data_handler=dHandler_mnist, problem_type=problem_type, train_epochs=100)
"""

"\nsave_best_models(global_best_list, global_best_index, 'best_models/mnist/alpha{}/scalarized_version/'.format(size_scaler), \n                 input_shape=input_shape, data_handler=dHandler_mnist, problem_type=problem_type, train_epochs=100)\n"

In [13]:
"""
save_best_models(global_best_list, global_best_index, 'best_models/cifar10/alpha{}/version2'.format(size_scaler), 
                 input_shape=input_shape, data_handler=dHandler_cifar, problem_type=problem_type, train_epochs=100)
"""

"\nsave_best_models(global_best_list, global_best_index, 'best_models/cifar10/alpha{}/version2'.format(size_scaler), \n                 input_shape=input_shape, data_handler=dHandler_cifar, problem_type=problem_type, train_epochs=100)\n"

In [14]:
"""
save_best_models(global_best_list, global_best_index, 'best_models/cmapss/alpha{}/'.format(size_scaler), 
                 input_shape=input_shape, data_handler=dhandler_cmaps, problem_type=problem_type, train_epochs=100, 
                 data_scaler=scaler, metrics=['rhs', 'rmse'], round=2)
"""

"\nsave_best_models(global_best_list, global_best_index, 'best_models/cmapss/alpha{}/'.format(size_scaler), \n                 input_shape=input_shape, data_handler=dhandler_cmaps, problem_type=problem_type, train_epochs=100, \n                 data_scaler=scaler, metrics=['rhs', 'rmse'], round=2)\n"